In [12]:
import pyedflib
import numpy as np
import matplotlib.pyplot as plt
import os


#Samples are represented in 16-bit 2's complement

# Get the directory where the script is located
script_dir = os.getcwd()
file_counter = 0
Files = []
badFiles = []
# Construct the full path to the file
file_path = os.path.join(script_dir, 'EDF', 'PD patient Frontal')
for filename in os.listdir(file_path):
    # Check if the file ends with .edf
    if filename.endswith('.edf'):
        file_counter = file_counter+1
        Files.append(filename)
for k in np.arange(file_counter):
    path = os.path.join(file_path, Files[k])
    try:
        f = pyedflib.EdfReader(path)
    except OSError:
        badFiles.append(Files[k])     
Files = [item for item in Files if item not in badFiles]
Files.reverse()
n = f.signals_in_file
n = n-9
number_of_samples = f.getNSamples()[0]
Nblocks = int((number_of_samples-250)/64)
TotalBlocks=(5*Nblocks)*n
fuller_data = np.ndarray(shape=(TotalBlocks, 64, 2))
BlockCount=0
multiplier = f.getPhysicalMaximum(0)/f.getDigitalMaximum(0)
f.close()
for index, name in enumerate(Files):
    print(name)
    path = os.path.join(file_path, name)
    f = pyedflib.EdfReader(path)
    number_of_samples = f.getNSamples()[0]
    Nblocks = int((number_of_samples-250)/64)
    sigbufs = np.zeros(number_of_samples)
    full_data64 = np.ndarray(shape=(Nblocks*n, 64, 2))
    signalList = []  
    BlockCount = BlockCount+Nblocks*n
    ran = np.ndarray(shape=(21, number_of_samples-250))
    for i in np.arange(n):
        sigbufs[:] = f.readSignal(i, digital=True)
        sigbufs_new = sigbufs[250:]
        #ran[i] = sigbufs_new
        signalList.append(sigbufs_new) 
        labels = np.zeros(number_of_samples-250)
        if name == "DP14.edf":
            sezStart = 79650
            sezEnd = 82250
            labels[sezStart:sezEnd] = 1
        elif name == "DP141.edf":
            sezStart = 103500
            sezEnd = 106500
            labels[sezStart:sezEnd] = 1
        elif name == "DP142.edf":
            sezStart = 223250
            sezEnd = 224750
            labels[sezStart:sezEnd] = 1
        elif name == "DP18.edf":
            sezStart = 93250
            sezEnd = 94000
            labels[sezStart:sezEnd] = 1
        for j in np.arange(Nblocks):
                full_data64[i*Nblocks+j,:,0] = signalList[i][j*64:(j+1)*64]
                full_data64[i*Nblocks+j,:,1] = labels[j*64:(j+1)*64]
    fuller_data[BlockCount-Nblocks*n:BlockCount] = full_data64
    f.close()
seizures = np.sum(fuller_data[:,:,1] == 1) 
normal = np.sum(fuller_data[:,:,1] == 0)
percentage_seizure = seizures/(seizures+normal)*100
print(fuller_data.shape)
print(percentage_seizure)

DP14.edf
DP18.edf
DP141.edf
DP142.edf
DP15.edf
(369495, 64, 2)
0.697108553566354


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_callbacks

NSTEPS = int(369495 * 0.8) // 128 

def pruneFunction(layer):
    pruning_params = {
        'pruning_schedule': sparsity.PolynomialDecay(
            initial_sparsity=0.0, final_sparsity=0.3, begin_step=NSTEPS * 2, end_step=NSTEPS * 10, frequency=NSTEPS
        )
    }
    if isinstance(layer, tf.keras.layers.Conv1D):
        return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
    if isinstance(layer, tf.keras.layers.Dense) and layer.name != 'output_dense':
        return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
    return layer

2024-08-05 22:10:16.936236: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 22:10:16.936288: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 22:10:16.937631: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-05 22:10:16.943216: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-05 22:10:18.036510: W tensorflow/compiler/tf2

In [2]:
def prd_loss_dig2phy(y_true, y_pred):
    y_true = (y_true)*multiplier
    y_pred = (y_pred)*multiplier
    rms_deviation = (tf.reduce_sum(tf.square(y_true - y_pred)))
    percentage_rmsd = tf.sqrt(rms_deviation/(tf.reduce_sum(tf.square(y_true))+tf.keras.backend.epsilon()))* 100
    return percentage_rmsd

In [3]:
import tensorflow.keras.backend as K

def weighted_mse_loss(y_true, y_pred):
    # y_true, y_pred shape: (batch_size, num_channels, signal_length)
    # label shape: (batch_size, )
    weight = 150
    # Extract the labels (0 or 1) from the last dimension of y_true
    labels = y_true[:, :, 1]
    # Remove the labels from y_true for loss calculation
    y_true_processed = y_true[:, :, 0]
    y_true_processed=y_true_processed[..., tf.newaxis]
    loss = K.mean(K.square(y_pred - y_true_processed))
    weighted_loss = loss * ((labels * (weight - 1)) + 1)
    return K.mean(weighted_loss)

In [4]:
def prd_loss_dig2phy_new(y_true, y_pred):
    y_true_processed = y_true[:, :, 0]
    y_true_processed = y_true_processed[..., tf.newaxis]
    y_true_processed = (y_true_processed)*multiplier
    y_pred = (y_pred)*multiplier
    rms_deviation = (tf.reduce_sum(tf.square(y_true_processed - y_pred)))
    percentage_rmsd = tf.sqrt(rms_deviation/(tf.reduce_sum(tf.square(y_true_processed))+tf.keras.backend.epsilon()))* 100
    return percentage_rmsd

In [14]:
import tensorflow as tf
from tensorflow import keras
from qkeras import quantizers
from qkeras import QConv1D, QDense, QActivation
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from qkeras import quantizers
from qkeras import QConv1D, QDense, QActivation
from qkeras.utils import load_qmodel
from keras.optimizers import Adam

# Define the encoder with quantization
input_ts = keras.layers.Input(shape=(64, 1), name="input_time_series")  # 64, 1

# First Conv1D layer with quantized activations
x = QConv1D(8, 3, 
            kernel_quantizer=quantizers.quantized_bits(12, 0, alpha=1), 
            bias_quantizer=quantizers.quantized_bits(12, 0, alpha=1),
            padding='same')(input_ts)  # 64, 16
x = QActivation(activation= quantizers.quantized_bits(28, 15, alpha=1))(x)  # Quantized activations to 16 bits
# Second Conv1D layer with quantized activations
x = QConv1D(8, 5, 
            kernel_quantizer=quantizers.quantized_bits(12, 0, alpha=1), 
            bias_quantizer=quantizers.quantized_bits(12, 0, alpha=1),
            padding='same')(x)  # 64, 16
x = QActivation(activation= quantizers.quantized_bits(28, 15, alpha=1))(x)
# Third Conv1D layer with quantized activations
x = QConv1D(4, 5, 
            kernel_quantizer=quantizers.quantized_bits(12, 0, alpha=1), 
            bias_quantizer=quantizers.quantized_bits(12, 0, alpha=1),
            padding='same', 
            strides=2)(x)  # 32, 8
x = QActivation(activation= quantizers.quantized_bits(28, 15, alpha=1))(x)
# Fourth Conv1D layer with quantized activations
x = QConv1D(4, 5, 
            kernel_quantizer=quantizers.quantized_bits(12, 0, alpha=1), 
            bias_quantizer=quantizers.quantized_bits(12, 0, alpha=1),
            padding='same', 
            strides=2)(x)  # 16, 4
x = QActivation(activation= quantizers.quantized_bits(28, 15, alpha=1))(x)
# Flatten layer
x = keras.layers.Flatten()(x)  # Flatten for Dense layer

# Dense layer with quantized activations
x = QDense(16, kernel_quantizer=quantizers.quantized_bits(12, 0, alpha=1), 
                 bias_quantizer=quantizers.quantized_bits(12, 0, alpha=1))(x)  # 16
encoded = keras.layers.Activation(activation='linear')(x)

# Define the model
encoder = keras.models.Model(input_ts, encoded, name="encoder")

# Print the model summary
encoder.summary()

# Define the decoder
encoded_input = keras.layers.Input(shape=(16,), name="encoded_input") # 16
x = keras.layers.Dense(16 * 4, activation='linear')(encoded_input) # 16 * 4
x = keras.layers.Reshape((16, 4))(x) # Reshape back to (16, 4)
x = keras.layers.Conv1DTranspose(8, 5, activation='linear', strides=2, padding='same')(x) # 16, 8
x = keras.layers.Conv1DTranspose(16, 5, activation='linear', strides=2, padding='same')(x) # 32, 16
x = keras.layers.Conv1DTranspose(8, 7, activation='linear', padding='same')(x) # 32, 16
decoded = keras.layers.Conv1DTranspose(1, 3, activation='linear', padding='same')(x) # 64, 1

decoder = keras.models.Model(encoded_input, decoded, name="decoder")


Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_time_series (InputLa  [(None, 64, 1)]           0         
 yer)                                                            
                                                                 
 q_conv1d (QConv1D)          (None, 64, 8)             32        
                                                                 
 q_activation (QActivation)  (None, 64, 8)             0         
                                                                 
 q_conv1d_1 (QConv1D)        (None, 64, 8)             328       
                                                                 
 q_activation_1 (QActivatio  (None, 64, 8)             0         
 n)                                                              
                                                                 
 q_conv1d_2 (QConv1D)        (None, 32, 4)             164 

In [15]:
prunedQenocder = tf.keras.models.clone_model(encoder, clone_function=pruneFunction)

ValueError: `prune_low_magnitude` can only prune an object of the following types: keras.models.Sequential, keras functional model, keras.layers.Layer, list of keras.layers.Layer. You passed an object of type: QConv1D.

In [ ]:
# Define the autoencoder
autoencoder_input = keras.layers.Input(shape=(64, 1), name="autoencoder_input")
encoded_ts = encoder(autoencoder_input)
decoded_ts = decoder(encoded_ts)
autoencoder = keras.models.Model(autoencoder_input, decoded_ts, name="autoencoder")

# Compile the autoencoder
autoencoder.compile(optimizer=Adam(learning_rate=0.00001), loss=weighted_mse_loss, metrics=[prd_loss_dig2phy_new])
weight_model = load_qmodel('quantised.h5', custom_objects={'prd_loss_dig2phy_new': prd_loss_dig2phy_new, 'weighted_mse_loss': weighted_mse_loss})
# Summary of the autoencoder
autoencoder.summary()
autoencoder.set_weights(weight_model.get_weights())
checkpoint_callback = ModelCheckpoint(
    filepath='inter1_quantised.h5',  # Path to save the model
    monitor='val_loss',        # Metric to monitor
    save_best_only=True,       # Save only the best model
    mode='min',                # Mode: minimize the monitored metric
    verbose=1                  # Print a message when saving the model
)
#features_train = fuller_data[:, :, 0]
# features_val = val_data[:, :, 0]

new_train, new_val = train_test_split(fuller_data, test_size=0.2, random_state=69)
features_val = new_val[:, :, 0]
features_val = features_val[..., tf.newaxis]
features_train = new_train[:, :, 0]
features_train = features_train[..., tf.newaxis]
pred = autoencoder.predict(features_val)
print(prd_loss_dig2phy(features_val, pred).numpy())
# Train the autoencoder
autoencoder.fit(features_train, new_train, epochs=30, batch_size=128, shuffle=True, callbacks=[checkpoint_callback], validation_data=(features_val, new_val))
